In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, log_loss, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from scipy.stats import mode
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import ElasticNet
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier


## Loading the dataset

In [2]:
df = pd.read_csv('pd_EEG_features.csv')
df.head()

,id,gender,PPE,EEG,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1


In [3]:
X = df.drop(columns=['id','class']) 
y = df['class']  

In [4]:
count0=0
count1=0
for i in y:
    if i==0:
        count0+=1
    else:
        count1+=1

print(count0,count1)

192 564


## Balancing the dataset

In [5]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
df_resampled = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled, columns=['class'])], axis=1)

In [6]:
df_resampled

,gender,PPE,EEG,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,1,0.852470,0.718260,0.572270,240,239,0.008064,0.000087,0.002180,0.000018,...,1.562000,2.644500,3.868600,4.210500,5.122100,4.462500,2.620200,3.000400,18.940500,1
1,1,0.766860,0.694810,0.539660,234,233,0.008258,0.000073,0.001950,0.000016,...,1.558900,3.610700,23.515500,14.196200,11.026100,9.508200,6.524500,6.343100,45.178000,1
2,1,0.850830,0.676040,0.589820,232,231,0.008340,0.000060,0.001760,0.000015,...,1.564300,2.330800,9.495900,10.745800,11.017700,4.806600,2.919900,3.149500,4.766600,1
3,0,0.411210,0.796720,0.592570,178,177,0.010858,0.000183,0.004190,0.000046,...,3.780500,3.566400,5.255800,14.040300,4.223500,4.685700,4.846000,6.265000,4.060300,1
4,0,0.327900,0.797820,0.530280,236,235,0.008162,0.002669,0.005350,0.000044,...,6.172700,5.841600,6.080500,5.762100,7.781700,11.689100,8.210300,5.055900,6.116400,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,0,0.846790,0.672417,0.462422,504,503,0.003835,0.000061,0.000767,0.000003,...,8.647863,4.877069,4.029061,3.721892,3.927724,4.420524,4.450813,4.594596,4.238312,0
1124,0,0.830738,0.696754,0.353928,440,439,0.004385,0.000032,0.000781,0.000003,...,54.538588,52.496380,36.303025,22.949557,15.209481,12.072220,14.473528,14.720977,29.266257,0
1125,0,0.822565,0.563505,0.413189,493,492,0.003914,0.000045,0.000547,0.000002,...,86.697327,58.214193,47.425700,39.920400,31.880980,26.469694,19.636002,10.795145,3.076334,0
1126,0,0.787599,0.609335,0.207256,594,593,0.003245,0.000033,0.000373,0.000001,...,188.246003,146.915555,79.392584,38.204836,31.944149,30.986111,13.733797,9.864585,13.058255,0


In [7]:
X = df_resampled.drop(columns=['class']) 
y = df_resampled['class']  

In [8]:
count0=0
count1=0
for i in y:
    if i==0:
        count0+=1
    else:
        count1+=1

print(count0,count1)

564 564


In [9]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### XGBoost

In [58]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

In [59]:
xg_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - xg_accuracy
y_pred_proba = xgb.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",xg_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9424778761061947
Precision: 0.9642857142857143
Recall: 0.9230769230769231
F1 Score: 0.9432314410480349
Error Rate: 0.05752212389380529
Binary Cross-Entropy Loss: 0.1126669994341628
Confusion Matrix:
[[105   4]
 [  9 108]]


### AdaBoost Classifier

In [60]:
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)


C:\Users\sudha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [62]:
AB_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - AB_accuracy
y_pred_proba = ada.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",AB_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9513274336283186
Precision: 0.9649122807017544
Recall: 0.9401709401709402
F1 Score: 0.9523809523809523
Error Rate: 0.04867256637168138
Binary Cross-Entropy Loss: 0.635890992699611
Confusion Matrix:
[[105   4]
 [  7 110]]


### CatBoost

In [63]:
catboost_classifier = CatBoostClassifier(n_estimators=100, random_state=42)
catboost_classifier.fit(X_train, y_train)
y_pred = catboost_classifier.predict(X_test)

Learning rate set to 0.081434
0:	learn: 0.6399365	total: 205ms	remaining: 20.3s
1:	learn: 0.5980812	total: 257ms	remaining: 12.6s
2:	learn: 0.5565662	total: 310ms	remaining: 10s
3:	learn: 0.5246087	total: 363ms	remaining: 8.71s
4:	learn: 0.4959610	total: 416ms	remaining: 7.9s
5:	learn: 0.4676046	total: 470ms	remaining: 7.36s
6:	learn: 0.4360529	total: 523ms	remaining: 6.95s
7:	learn: 0.4145788	total: 577ms	remaining: 6.64s
8:	learn: 0.3943257	total: 629ms	remaining: 6.36s
9:	learn: 0.3776635	total: 681ms	remaining: 6.13s
10:	learn: 0.3634121	total: 734ms	remaining: 5.94s
11:	learn: 0.3507654	total: 786ms	remaining: 5.76s
12:	learn: 0.3354181	total: 838ms	remaining: 5.61s
13:	learn: 0.3226549	total: 891ms	remaining: 5.47s
14:	learn: 0.3108936	total: 945ms	remaining: 5.35s
15:	learn: 0.3023147	total: 1s	remaining: 5.25s
16:	learn: 0.2917577	total: 1.05s	remaining: 5.15s
17:	learn: 0.2820982	total: 1.11s	remaining: 5.05s
18:	learn: 0.2702501	total: 1.16s	remaining: 4.95s
19:	learn: 0.2611

In [64]:
catboost_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - catboost_accuracy
y_pred_proba = catboost_classifier.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",catboost_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9690265486725663
Precision: 0.9824561403508771
Recall: 0.9572649572649573
F1 Score: 0.9696969696969697
Error Rate: 0.030973451327433676
Binary Cross-Entropy Loss: 0.1265617108437408
Confusion Matrix:
[[107   2]
 [  5 112]]


### GBM

In [65]:
gbm = GradientBoostingClassifier(
    n_estimators=100,      
    learning_rate=0.05,
    max_depth=5,          
    max_features=0.9,
    random_state=42
)
gbm.fit(X_train, y_train)

y_pred_prob = gbm.predict_proba(X_test)[:, 1]
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred_prob]


In [66]:
gbm_accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - gbm_accuracy
y_pred_proba = gbm.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",gbm_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9424778761061947
Precision: 0.9824561403508771
Recall: 0.9572649572649573
F1 Score: 0.9696969696969697
Error Rate: 0.05752212389380529
Binary Cross-Entropy Loss: 0.1260870345126343
Confusion Matrix:
[[107   2]
 [  5 112]]


### LightGBM

In [57]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}
lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)
y_pred = lgbm.predict(X_test, num_iteration=lgbm.best_iteration)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]


[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019593 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [58]:
lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
error_rate = 1 - lgbm_accuracy
y_pred_proba = 1 / (1 + np.exp(-y_pred))
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred_binary)

print("Accuracy:",lgbm_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9734513274336283
Precision: 0.9911504424778761
Recall: 0.9572649572649573
F1 Score: 0.9739130434782609
Error Rate: 0.026548672566371723
Binary Cross-Entropy Loss: 0.5305929448832605
Confusion Matrix:
[[108   1]
 [  5 112]]


### Ridge Classifier

In [59]:
ridge_classifier = RidgeClassifier(
    alpha=1.0,  
    random_state=42
)
ridge_classifier.fit(X_train, y_train)
y_pred = ridge_classifier.predict(X_test)
y_pred_binary = (y_pred >= 0.5).astype(int)

C:\Users\sudha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=4.5274e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [60]:
ridge_accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - ridge_accuracy
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",ridge_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8982300884955752
Precision: 0.97
Recall: 0.8290598290598291
F1 Score: 0.8940092165898618
Error Rate: 0.10176991150442483
Confusion Matrix:
[[106   3]
 [ 20  97]]


### ElasticNet 

In [61]:
elastic_net_classifier = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)
elastic_net_classifier.fit(X_train, y_train)
y_pred = elastic_net_classifier.predict(X_test)
y_pred_binary = (y_pred >= 0.5).astype(int)


C:\Users\sudha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.936e+01, tolerance: 2.255e-02
  model = cd_fast.enet_coordinate_descent(


In [62]:
elastic_net_accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
error_rate = 1 - elastic_net_accuracy
conf_matrix = confusion_matrix(y_test, y_pred_binary)

print("Accuracy:",elastic_net_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8761061946902655
Precision: 0.908256880733945
Recall: 0.8461538461538461
F1 Score: 0.8761061946902655
Error Rate: 0.12389380530973448
Confusion Matrix:
[[99 10]
 [18 99]]


### RandomForest

In [63]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)


In [64]:
rf_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - rf_accuracy
y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",rf_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9380530973451328
Precision: 0.963963963963964
Recall: 0.9145299145299145
F1 Score: 0.9385964912280702
Error Rate: 0.06194690265486724
Binary Cross-Entropy Loss: 0.23266725444465847
Confusion Matrix:
[[105   4]
 [ 10 107]]


### ExtraTrees

In [67]:
et_classifier = ExtraTreesClassifier(n_estimators=100, random_state=42)
et_classifier.fit(X_train, y_train)
y_pred = et_classifier.predict(X_test)

In [68]:
et_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - et_accuracy
y_pred_proba = et_classifier.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",et_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9469026548672567
Precision: 0.981651376146789
Recall: 0.9145299145299145
F1 Score: 0.9469026548672567
Error Rate: 0.053097345132743334
Binary Cross-Entropy Loss: 0.18748306756059469
Confusion Matrix:
[[107   2]
 [ 10 107]]


### GMM

In [67]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
gmm = GaussianMixture(n_components=2, random_state=42)
gmm.fit(X_scaled)
cluster_labels = gmm.predict(X_scaled)

def map_cluster_labels(true_labels, cluster_labels):
    new_labels = np.zeros_like(cluster_labels)
    for i in range(np.max(cluster_labels) + 1):
        mask = (cluster_labels == i)
        new_labels[mask] = mode(true_labels[mask])[0]
    return new_labels

true_labels = y  
mapped_labels = map_cluster_labels(true_labels, cluster_labels)


In [68]:
gmm_accuracy = accuracy_score(true_labels, mapped_labels)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - gmm_accuracy
y_pred_proba = gmm.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",gmm_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.7313829787234043
Precision: 0.981651376146789
Recall: 0.9145299145299145
F1 Score: 0.9469026548672567
Error Rate: 0.2686170212765957
Binary Cross-Entropy Loss: 18.659767462507553
Confusion Matrix:
[[107   2]
 [ 10 107]]


C:\Users\sudha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(


### Accuracy on UNBALANCED data

In [69]:
print(f'XGBoost Accuracy: {xg_accuracy}')
print(f'AdaBoost Accuracy: {AB_accuracy}')
print(f"CatBoost Accuracy:{catboost_accuracy}")
print(f'GBM Accuracy: {gbm_accuracy}')
print(f'LightGBM Accuracy: {lgbm_accuracy}')
print(f'Ridge Accuracy: {ridge_accuracy}')
print(f'ElasticNet Accuracy: {elastic_net_accuracy}')
print(f'RandomForest Accuracy: {rf_accuracy}')
print(f'ExtraTrees Accuracy: {et_accuracy}')
print(f'GMM Accuracy: {gmm_accuracy}')

XGBoost Accuracy: 0.9424778761061947
AdaBoost Accuracy: 0.9513274336283186
CatBoost Accuracy:0.9690265486725663
GBM Accuracy: 0.9424778761061947
LightGBM Accuracy: 0.9734513274336283
Ridge Accuracy: 0.8982300884955752
ElasticNet Accuracy: 0.8761061946902655
RandomForest Accuracy: 0.9380530973451328
ExtraTrees Accuracy: 0.9469026548672567
GMM Accuracy: 0.7313829787234043


In [ ]:
### Enhancing LightGBM

In [11]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}
lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)
y_pred = lgbm.predict(X_test, num_iteration=lgbm.best_iteration)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:",lgbm_accuracy)


[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.051448 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy: 0.9734513274336283


In [12]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'num_leaves': [15, 31, 50],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'feature_fraction': [0.6, 0.8, 1.0]
}

# Initialize the LightGBM classifier
lgbm = LGBMClassifier(objective='binary', metric='binary_error', boosting_type='gbdt')

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_lgbm = grid_search.best_estimator_

# Evaluate the best model on the test set
accuracy = best_lgbm.score(X_test, y_test)
print("Best Model Accuracy:", accuracy)



Fitting 5 folds for each of 81 candidates, totalling 405 fits


KeyboardInterrupt: 

In [16]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

# Define the objective function to be maximized (in this case, accuracy)
def lgbm_cv(num_leaves, learning_rate, n_estimators, feature_fraction):
    params = {
        'objective': 'binary',
        'metric': 'binary_error',
        'boosting_type': 'gbdt',
        'num_leaves': int(num_leaves),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'feature_fraction': feature_fraction
    }
    lgbm = LGBMClassifier(**params)
    accuracy = cross_val_score(lgbm, X_train, y_train, cv=5, scoring='accuracy').mean()
    return accuracy

# Define the parameter space for Bayesian optimization
pbounds = {
    'num_leaves': (15, 50),
    'learning_rate': (0.01, 0.1),
    'n_estimators': (100, 300),
    'feature_fraction': (0.6, 1.0)
}

# Initialize BayesianOptimization object
optimizer = BayesianOptimization(
    f=lgbm_cv,
    pbounds=pbounds,
    random_state=42,
)

# Perform optimization
optimizer.maximize(init_points=5, n_iter=10)

# Get the best hyperparameters
best_params = optimizer.max['params']
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_lgbm = LGBMClassifier(objective='binary', metric='binary_error', boosting_type='gbdt', **best_params)


|   iter    |  target   | featur... | learni... | n_esti... | num_le... |
-------------------------------------------------------------------------
[LightGBM] [Warning] feature_fraction is set=0.749816047538945, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.749816047538945
[LightGBM] [Warning] feature_fraction is set=0.749816047538945, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.749816047538945
[LightGBM] [Info] Number of positive: 357, number of negative: 364
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029814 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 180473
[LightGBM] [Info] Number of data points in the train set: 721, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495146 -> initscore=-0.019418
[LightGBM] [Info] Start training from score -0.019418
[LightGBM] [Warning] No further splits with positive gain

In [ ]:
best_lgbm.fit(X_train, y_train)

# Evaluate the model on the test set
accuracy = best_lgbm.score(X_test, y_test)
print("Best Model Accuracy:", accuracy)


In [18]:
# Convert n_estimators to integer
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['num_leaves'] = int(best_params['num_leaves'])

# Train the model with the best hyperparameters
best_lgbm = LGBMClassifier(objective='binary', metric='binary_error', boosting_type='gbdt', **best_params)
best_lgbm.fit(X_train, y_train)


[LightGBM] [Warning] feature_fraction is set=0.7216968971838151, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7216968971838151
[LightGBM] [Warning] feature_fraction is set=0.7216968971838151, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7216968971838151
[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.072943 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(feature_fraction=0.7216968971838151,
               learning_rate=0.05722807884690141, metric='binary_error',
               n_estimators=186, num_leaves=25, objective='binary')

In [19]:
# Evaluate the model on the test set
accuracy = best_lgbm.score(X_test, y_test)
print("Best Model Accuracy:", accuracy)

[LightGBM] [Warning] feature_fraction is set=0.7216968971838151, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7216968971838151
Best Model Accuracy: 0.9734513274336283


In [22]:
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Define search space for hyperparameters
space = {
    'num_leaves': hp.quniform('num_leaves', 2, 50, 1),
    'learning_rate': hp.loguniform('learning_rate', -3, -0.0001),
    'feature_fraction': hp.uniform('feature_fraction', 0.6, 1.0)
}

# Define objective function for optimization
def objective(params):
    params['num_leaves'] = int(params['num_leaves'])
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'binary'
    params['metric'] = 'binary_error'
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    model = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval, verbose_eval=False)
    
    y_pred = model.predict(X_test)
    y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
    
    accuracy = accuracy_score(y_test, y_pred_binary)
    
    return -accuracy  # Negative because Hyperopt minimizes the objective

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform hyperparameter optimization
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

# Convert the best parameters to integers
best['num_leaves'] = int(best['num_leaves'])

# Train the model with the best hyperparameters
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

best_lgbm = lgb.train(best, lgb_train, num_boost_round=100, valid_sets=lgb_eval, verbose_eval=False)

# Evaluate the model on the test set
y_pred = best_lgbm.predict(X_test)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", lgbm_accuracy)


  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: train() got an unexpected keyword argument 'verbose_eval'



  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]


TypeError: train() got an unexpected keyword argument 'verbose_eval'

In [23]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'optimizer': 'optimal'  # Adding the optimizer
}

lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)
y_pred = lgbm.predict(X_test, num_iteration=lgbm.best_iteration)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", lgbm_accuracy)


[LightGBM] [Warning] Unknown parameter: optimizer
[LightGBM] [Warning] Unknown parameter: optimizer
[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.042766 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Warning] Unknown parameter: optimizer
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy: 0.9734513274336283


In [24]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from hyperopt import hp, fmin, tpe, Trials

# Define the search space for hyperparameters
space = {
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'boosting_type': hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    # Add more hyperparameters here as needed
}

# Define the objective function to minimize (in this case, negative accuracy)
def objective(params):
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval, verbose_eval=False)
    y_pred = lgbm.predict(X_test)
    y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
    lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
    
    return -lgbm_accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# Retrieve the best parameters
best_params = {key: value for key, value in best.items()}
best_params['boosting_type'] = ['gbdt', 'dart', 'goss'][best_params['boosting_type']]

print("Best Parameters:", best_params)

# Train the model with the best parameters
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
lgbm = lgb.train(best_params, lgb_train, num_boost_round=100, valid_sets=lgb_eval, verbose_eval=False)

# Evaluate the model
y_pred = lgbm.predict(X_test)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
print("Final Accuracy:", lgbm_accuracy)


  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

job exception: train() got an unexpected keyword argument 'verbose_eval'



  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]


TypeError: train() got an unexpected keyword argument 'verbose_eval'

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from hyperopt import hp, fmin, tpe, Trials

# Define the search space for hyperparameters
space = {
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),  # Ensure num_leaves is sampled as an integer
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'boosting_type': hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    # Add more hyperparameters here as needed
}

# Define the objective function to minimize (in this case, negative accuracy)
def objective(params):
    params['num_leaves'] = int(params['num_leaves'])  # Convert num_leaves to integer
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)
    y_pred = lgbm.predict(X_test)
    y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
    lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
    
    return -lgbm_accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# Retrieve the best parameters
best_params = {key: value for key, value in best.items()}
best_params['boosting_type'] = ['gbdt', 'dart', 'goss'][best_params['boosting_type']]

print("Best Parameters:", best_params)

# Train the model with the best parameters
best_params['num_leaves'] = int(best_params['num_leaves'])  # Convert num_leaves to integer
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
lgbm = lgb.train(best_params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)

# Evaluate the model
y_pred = lgbm.predict(X_test)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
print("Final Accuracy:", lgbm_accuracy)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050999 seconds.                
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489                                                                                    
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753                            
[LightGBM] [Info] Start training from score 0.495565                                                                   
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                            

In [10]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}
lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)
y_pred = lgbm.predict(X_test, num_iteration=lgbm.best_iteration)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:",lgbm_accuracy)


[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043288 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy: 0.9734513274336283


In [11]:
import itertools

# Define possible values for each parameter
num_leaves_values = [20, 50, 100, 150, 200]
learning_rate_values = [0.01, 0.05, 0.1, 0.2]
feature_fraction_values = [0.5, 0.7, 0.8, 0.9, 1.0]

# Generate all combinations
parameter_combinations = list(itertools.product(num_leaves_values, learning_rate_values, feature_fraction_values))

# Print the combinations
for params in parameter_combinations:
    print(params)


(20, 0.01, 0.5)
(20, 0.01, 0.7)
(20, 0.01, 0.8)
(20, 0.01, 0.9)
(20, 0.01, 1.0)
(20, 0.05, 0.5)
(20, 0.05, 0.7)
(20, 0.05, 0.8)
(20, 0.05, 0.9)
(20, 0.05, 1.0)
(20, 0.1, 0.5)
(20, 0.1, 0.7)
(20, 0.1, 0.8)
(20, 0.1, 0.9)
(20, 0.1, 1.0)
(20, 0.2, 0.5)
(20, 0.2, 0.7)
(20, 0.2, 0.8)
(20, 0.2, 0.9)
(20, 0.2, 1.0)
(50, 0.01, 0.5)
(50, 0.01, 0.7)
(50, 0.01, 0.8)
(50, 0.01, 0.9)
(50, 0.01, 1.0)
(50, 0.05, 0.5)
(50, 0.05, 0.7)
(50, 0.05, 0.8)
(50, 0.05, 0.9)
(50, 0.05, 1.0)
(50, 0.1, 0.5)
(50, 0.1, 0.7)
(50, 0.1, 0.8)
(50, 0.1, 0.9)
(50, 0.1, 1.0)
(50, 0.2, 0.5)
(50, 0.2, 0.7)
(50, 0.2, 0.8)
(50, 0.2, 0.9)
(50, 0.2, 1.0)
(100, 0.01, 0.5)
(100, 0.01, 0.7)
(100, 0.01, 0.8)
(100, 0.01, 0.9)
(100, 0.01, 1.0)
(100, 0.05, 0.5)
(100, 0.05, 0.7)
(100, 0.05, 0.8)
(100, 0.05, 0.9)
(100, 0.05, 1.0)
(100, 0.1, 0.5)
(100, 0.1, 0.7)
(100, 0.1, 0.8)
(100, 0.1, 0.9)
(100, 0.1, 1.0)
(100, 0.2, 0.5)
(100, 0.2, 0.7)
(100, 0.2, 0.8)
(100, 0.2, 0.9)
(100, 0.2, 1.0)
(150, 0.01, 0.5)
(150, 0.01, 0.7)
(150, 0.01, 0.8)

In [13]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import itertools

# Define possible values for each parameter
num_leaves_values = [20, 50, 100, 150, 200]
learning_rate_values = [0.01, 0.05, 0.1, 0.2]
feature_fraction_values = [0.5, 0.7, 0.8, 0.9, 1.0]

best_accuracy = 0.0
best_params = None

# Iterate over all combinations
for num_leaves, learning_rate, feature_fraction in itertools.product(num_leaves_values, learning_rate_values, feature_fraction_values):
    params = {
        'objective': 'binary',
        'metric': 'binary_error',
        'boosting_type': 'gbdt',
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'feature_fraction': feature_fraction
    }
    
    # Train the model
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)
    
    # Evaluate accuracy
    y_pred = lgbm.predict(X_test, num_iteration=lgbm.best_iteration)
    y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
    accuracy = accuracy_score(y_test, y_pred_binary)
    
    # Update best accuracy and parameters if necessary
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)


[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026074 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.037962 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Info] 

In [15]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 20,
    'learning_rate': 0.2,
    'feature_fraction': 0.9
}
lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)
y_pred = lgbm.predict(X_test, num_iteration=lgbm.best_iteration)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:",lgbm_accuracy)


[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.083105 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [24]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from scipy.optimize import minimize

# Train the LightGBM model with fixed hyperparameters
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 20,
    'learning_rate': 0.2,
    'feature_fraction': 0.9
}
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)

# Define the objective function to maximize (e.g., accuracy)
def objective_function(threshold):
    y_pred = lgbm.predict(X_test)
    y_pred_binary = (y_pred > threshold).astype(int)
    accuracy = accuracy_score(y_test, y_pred_binary)
    return -accuracy  # We want to maximize accuracy, so we minimize negative accuracy

# Use an optimizer to find the best threshold
initial_guess = 0.45  # Initial guess for the threshold
result = minimize(objective_function, initial_guess, method='Nelder-Mead')

# Retrieve the best threshold found
best_threshold = result.x[0]
print("Best Threshold:", best_threshold)

# Evaluate the model with the best threshold
y_pred = lgbm.predict(X_test)
y_pred_binary = (y_pred > best_threshold).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy with Best Threshold:", accuracy)


[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043152 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [54]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# Define regularization parameters
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 20,
    'learning_rate': 0.2,
    'feature_fraction': 0.9,
    'lambda_l1': 1,  # L1 regularization
    'lambda_l2': 1   # L2 regularization
}

# Train the LightGBM model with regularization
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)

# Evaluate the model
y_pred = lgbm.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)  # Assuming threshold of 0.5
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)


[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020825 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [52]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from scipy.optimize import minimize

# Train the LightGBM model with fixed hyperparameters
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 20,
    'learning_rate': 0.2,
    'feature_fraction': 0.9,
    'lambda_l1': 0.0001,  # L1 regularization
    'lambda_l2': 0.0001   # L2 regularization
}
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)

# Define the objective function to maximize (e.g., accuracy)
def objective_function(threshold):
    y_pred = lgbm.predict(X_test)
    y_pred_binary = (y_pred > threshold).astype(int)
    accuracy = accuracy_score(y_test, y_pred_binary)
    return -accuracy  # We want to maximize accuracy, so we minimize negative accuracy

# Use an optimizer to find the best threshold
initial_guess = 0.2  # Initial guess for the threshold
result = minimize(objective_function, initial_guess, method='Nelder-Mead')

# Retrieve the best threshold found
best_threshold = result.x[0]
print("Best Threshold:", best_threshold)

# Evaluate the model with the best threshold
y_pred = lgbm.predict(X_test)
y_pred_binary = (y_pred > best_threshold).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy with Best Threshold:", accuracy)


[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [55]:
from sklearn.ensemble import VotingClassifier
